## 네이버 금융 
- selenium 
- iframe 
    - HTML 안에 또 다른 HTML 

In [30]:
%reset

Once deleted, variables cannot be recovered. Proceed (y/[n])? y


In [31]:
import pandas as pd 
import time 
from selenium import webdriver

In [32]:
url = "https://finance.naver.com/marketindex/"
driver = webdriver.Chrome("../driver/chromedriver") 
driver.get(url)

In [4]:
# 환전 고시 환율 
contents = driver.find_elements_by_css_selector("#exchangeList > li > a")
contents[0].find_element_by_css_selector(".value").text

'1,178.00'

In [5]:
# 주요뉴스 제목 
news = driver.find_elements_by_css_selector("#content > div.section_news > div > ul > li:nth-child(1) > p > a")
news[0].text

'한은 “국제유가 하반기 70달러 내외 유지'

In [6]:
# 밑 페이지에 있는, 매매기준율 
driver.find_elements_by_css_selector("body > div > table > tbody > tr")

[]

## iframe 

In [7]:
# selenium을 이용한 방법 

In [33]:
# iframe 태그 지정 

iframe = driver.find_element_by_css_selector("#frame_ex1")
# driver.find_element_by_id("frame_ex1")

In [34]:
# iframe 태그 이동
driver.switch_to_frame(iframe)

/var/folders/8c/jb57288j6xlb3s_tkmys1kj00000gn/T/ipykernel_4840/630311945.py:2: DeprecationWarning: use driver.switch_to.frame instead
  driver.switch_to_frame(iframe)


In [36]:
# 환전 고시 환율 
contents = driver.find_elements_by_css_selector("#exchangeList > li > a")
contents[0].find_element_by_css_selector(".value").text

IndexError: list index out of range

In [37]:
driver.find_element_by_css_selector(".sale").text

'1,178.00'

In [39]:
# 밑 페이지에 있는, 매매기준율 
contents = driver.find_elements_by_css_selector("body > div > table > tbody > tr")
contents[0].find_element_by_css_selector(".sale").text

'1,178.00'

In [42]:
datas = [] 

for content in contents:
    title = content.find_element_by_css_selector(".tit > a").text
    sale = content.find_element_by_css_selector(".sale").text
    link = content.find_element_by_css_selector(".tit > a").get_attribute("href")
    datas.append({
        "title": title, 
        "sale": sale,
        "link": link
    })
    
df = pd.DataFrame(datas)
df.to_excel("./sel_naver_finance.xlsx", encoding="utf-8")
df

MaxRetryError: HTTPConnectionPool(host='127.0.0.1', port=54252): Max retries exceeded with url: /session/65cfa7578b988222fc83bf27f33df004/element/17f7e1a7-5b3c-4df7-a23d-17d66365b7f4/element (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x1689ca040>: Failed to establish a new connection: [Errno 61] Connection refused'))

In [41]:
driver.quit()

In [43]:
# request를 이용한 방법 

In [44]:
import requests
from bs4 import BeautifulSoup

In [45]:
url = "https://finance.naver.com/marketindex/exchangeList.naver"
response = requests.get(url)
response

<Response [200]>

In [46]:
response.content

b'<html lang=\'ko\'>\n<head>\n<title>\xb3\xd7\xc0\xcc\xb9\xf6 \xb1\xdd\xc0\xb6</title>\n<meta http-equiv=\'Content-Type\' content=\'text/html; charset=euc-kr\'>\n<meta http-equiv="Content-Script-Type" content="text/javascript">\n<meta http-equiv="Content-Style-Type" content="text/css">\n<link rel="stylesheet" type="text/css" href="https://ssl.pstatic.net/imgstock/static.pc/20210916165954/css/finance.css">\n\n<script language="javascript">document.domain="naver.com";</script>\n<script type="text/javascript" src="https://ssl.pstatic.net/imgstock/static.pc/20210916165954/js/info/jindo.min.ns.1.5.3.euckr.js"></script>\n<script type="text/javascript" src="https://ssl.pstatic.net/imgstock/static.pc/20210916165954/js/lcslog.js"></script>\n</head>\n<body>\n<div class="tbl_area">\n\t<table border="1" class="tbl_exchange" summary="\xc8\xaf\xc0\xfc \xb0\xed\xbd\xc3 \xc8\xaf\xc0\xb2 \xb8\xae\xbd\xba\xc6\xae">\n\t<caption>\xc8\xaf\xc0\xfc \xb0\xed\xbd\xc3 \xc8\xaf\xc0\xb2</caption>\n\t<colgroup>\n\

In [48]:
soup = BeautifulSoup(response.content, "html.parser")
print(soup.prettify())

<html lang="ko">
 <head>
  <title>
   네이버 금융
  </title>
  <meta content="text/html; charset=utf-8" http-equiv="Content-Type"/>
  <meta content="text/javascript" http-equiv="Content-Script-Type"/>
  <meta content="text/css" http-equiv="Content-Style-Type"/>
  <link href="https://ssl.pstatic.net/imgstock/static.pc/20210916165954/css/finance.css" rel="stylesheet" type="text/css"/>
  <script language="javascript">
   document.domain="naver.com";
  </script>
  <script src="https://ssl.pstatic.net/imgstock/static.pc/20210916165954/js/info/jindo.min.ns.1.5.3.euckr.js" type="text/javascript">
  </script>
  <script src="https://ssl.pstatic.net/imgstock/static.pc/20210916165954/js/lcslog.js" type="text/javascript">
  </script>
 </head>
 <body>
  <div class="tbl_area">
   <table border="1" class="tbl_exchange" summary="환전 고시 환율 리스트">
    <caption>
     환전 고시 환율
    </caption>
    <colgroup>
     <col width="162"/>
     <col width="92"/>
     <col width="92"/>
     <col width="92"/>
     <col widt

In [49]:
soup.select_one(".sale").text

'1,178.00'

In [52]:
contents = soup.select("tbody > tr") # 부모 태그 
contents[2].select_one(".sale").text

'1,066.50'

In [53]:
for content in contents:
    print(content.select_one(".sale").text)

1,178.00
1,382.44
1,066.50
182.11
151.30
42.48
1,613.21
3,059.66
928.99
1,277.03
136.45
855.88
829.31
54.42
1.50
133.26
0.41
368.11
185.90
137.28
314.07
3,913.62
3,125.08
320.71
1,661.50
75.03
35.27
870.95
281.25
8.28
320.61
2.77
870.95
15.97
6.95
13.82
23.24
58.47
222.12
5.17
78.71
16.15
3.87
299.77
